In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

<IPython.core.display.Javascript object>

In [2]:
#Accessing the Twitter API and retrieving tweets
import tweepy 

# Data manipulation and analysis
import pandas as pd 
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import  word_tokenize
from collections import Counter
from nltk.stem import WordNetLemmatizer
import re

#Visualisation
import plotly
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
import matplotlib.pyplot as plt

#Sentiment Analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from textblob import TextBlob

# feature extraction in NLP tasks
from sklearn.feature_extraction.text import TfidfVectorizer

# Clustering
from sklearn.cluster import KMeans

# Warning
import warnings
warnings.filterwarnings("ignore")

# Twitter's API Authentication and data collection

## Consumer_key and access_key generated from Twitter's developer account

In [3]:
CONSUMER_KEY = 'uB1vqJDt3bbJJQ6NiC1a1UzH3'
CONSUMER_SECRET = 'CaIYM9P0OLWjsJILw4RdjdB2rEae8kmXH4lP3ufY2Nm8ApT8EY'
ACCESS_KEY = '2980999297-07ApaDzXhuJZpBB6c9RrtiN69IFtzEZvv1H3t6w'
ACCESS_SECRET = 'jvsl9yiXNzZsYr1UBQaHDGJBKWfXiMVnmZtZEc1alE1eg'

## Initialize authentication and API object for accessing the Twitter API

In [4]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api = tweepy.API(auth)

## Collect relevant tweets based on a query and store them in a pandas DataFrame.

In [16]:
query = "#datascience OR #dataanalysis OR #dataengineer"

tweets = []

# Use the Cursor object to search for tweets
for tweet in tweepy.Cursor(api.search_tweets, q=query, lang='en').items(1000):
    # Store relevant data in a dictionary
    tweet_data = {
        'text': tweet.text,
        'created_at': tweet.created_at,
        'user_screen_name': tweet.user.screen_name,
        'retweet_count': tweet.retweet_count,
        'favorite_count': tweet.favorite_count,
        'hashtags': [hashtag['text'] for hashtag in tweet.entities['hashtags']]
    }
    # Append the dictionary to the list of tweets
    tweets.append(tweet_data)
    
# Create a DataFrame from the list of tweets
df = pd.DataFrame(tweets)

### Display the DataFrame

In [17]:
df.head()

,text,created_at,user_screen_name,retweet_count,favorite_count,hashtags
0,RT @DataScienceDojo: An overview of machine le...,2023-03-12 14:49:51+00:00,r_rancio,57,0,"[MachineLearning, Overview, DataScience]"
1,RT @gp_pulipaka: What is Data Science? #BigDat...,2023-03-12 14:49:47+00:00,leeziqiang0010,136,0,"[BigData, Analytics, DataScience, AI, MachineL..."
2,Really cool! I just reached 1500 subscribers o...,2023-03-12 14:49:40+00:00,MrAstroThomas,0,0,"[Space, Science, Python]"
3,RT @WestArt_Factory: Top data #Analytics tools...,2023-03-12 14:49:26+00:00,alexanderyamir1,41,0,"[Analytics, MachineLearning, DataScience, SQL,..."
4,RT @BDAnalyticsnews: How to become a data scie...,2023-03-12 14:49:15+00:00,BueleTejada,77,0,"[ArtificialIntelligence, AI, ML, DataScience, ..."


## Data cleaning and preprocessing on the tweets data

In [18]:
# Drop rows where the 'text' column is empty
df = df[df['text'].notna()]

# Convert 'created_at' column to datetime object
df['created_at'] = pd.to_datetime(df['created_at'])

# Drop rows where the 'user_screen_name' column is empty
df = df[df['user_screen_name'].notna()]

# Convert 'hashtags' column to string and fill empty values with an empty string
df['hashtags'] = df['hashtags'].fillna('').astype(str)

In [19]:
df.head()

,text,created_at,user_screen_name,retweet_count,favorite_count,hashtags
0,RT @DataScienceDojo: An overview of machine le...,2023-03-12 14:49:51+00:00,r_rancio,57,0,"['MachineLearning', 'Overview', 'DataScience']"
1,RT @gp_pulipaka: What is Data Science? #BigDat...,2023-03-12 14:49:47+00:00,leeziqiang0010,136,0,"['BigData', 'Analytics', 'DataScience', 'AI', ..."
2,Really cool! I just reached 1500 subscribers o...,2023-03-12 14:49:40+00:00,MrAstroThomas,0,0,"['Space', 'Science', 'Python']"
3,RT @WestArt_Factory: Top data #Analytics tools...,2023-03-12 14:49:26+00:00,alexanderyamir1,41,0,"['Analytics', 'MachineLearning', 'DataScience'..."
4,RT @BDAnalyticsnews: How to become a data scie...,2023-03-12 14:49:15+00:00,BueleTejada,77,0,"['ArtificialIntelligence', 'AI', 'ML', 'DataSc..."


In [20]:
df.to_csv('cleaned_tweets.csv', index=False)